In [104]:
from abc import ABC, abstractmethod
import time

class FibonacciFunc(ABC):
    def __init__(self):
        self.__steps = 0

    @abstractmethod
    def fib(self, n):
        pass

    def __getattribute__(self, __name):
        original = super().__getattribute__(__name)
        if __name in ['fib']:
            def __wrapped(*args, **kwargs):
                self.count_iteration()
                return original(*args, **kwargs)
            return __wrapped
        return original

    def count_iteration(self):
        self.__steps += 1

    def test(self, n):
        self.__steps = 0
        
        assert(self.fib(0) == 0)
        assert(self.fib(1) == 1)
        assert(self.fib(2) == 1)

        start = time.time()
        r = self.fib(n)
        end = time.time()
        diff = end - start

        print(f"go fib({n})={r} in {diff * 1000 * 1000}us with {self.__steps} steps")

def test_fib(n=300):
    def _wrapper(cls):
        c = cls()
        c.test(n)
    return _wrapper

In [87]:
""" Naive Recursive Solution
============================

Issues:
- Really slow
- Recusrive limitations via depth
"""

@test_fib(20)
class NaiveRecursiveFib(FibonacciFunc):
    def fib(self, n):
        if n in [0, 1]:
            return n
        return self.fib(n - 1) + self.fib(n - 2)

go fib(20)=6765 in 15032.052993774414us with 21896 steps


In [97]:
""" Cached Recursive Solution
=============================

Issues:
- Recusrive limitations via depth
- High memory usage to to full cache table
"""

@test_fib(300)
class CachedRecursiveFib(FibonacciFunc):
    def fib(self, n, cache={0: 0, 1: 1}):
        if n in cache:
            return cache[n]
        r = self.fib(n - 1, cache) + self.fib(n - 2, cache)
        cache[n] = r
        return r

go fib(300)=222232244629420445529739893461909967206666939096499764990979600 in 1181.3640594482422us with 602 steps


In [109]:
""" Iterative Tuple Solution
============================

Issues:
- Only one step at a time, no shortcuts
"""

@test_fib(300)
class IterativeTupleSolution(FibonacciFunc):
    def fib(self, n):
        if n in [0, 1]:
            return n
        r = (0, 1)
        for i in range(2, n + 1):
            self.count_iteration()
            r = (r[1], r[0] + r[1])
        return r[1]

go fib(300)=222232244629420445529739893461909967206666939096499764990979600 in 0.0us with 304 steps


In [111]:
""" Recursive Fast Doubling Solution
==========================

This is the fastest and still a simple solution to implement.

Issues:
- Recusrive limitations via depth

Improvements:
- Uses a lot less steps
"""

@test_fib(300)
class RecusriveFastDoublingFib(FibonacciFunc):
    def _fib(self, n):
        self.count_iteration()
        if n == 0:
            return (0, 1)
        else:
            a, b = self._fib(n // 2)
            c = a * (b * 2 - a)
            d = a * a + b * b
            if n % 2 == 0:
                return (c, d)
            else:
                return (d, c + d)
    def fib(self, n):
        return self._fib(n)[0]

go fib(300)=222232244629420445529739893461909967206666939096499764990979600 in 0.0us with 20 steps
